In [ ]:
import numpy as np
import open3d as o3d
from pathlib import Path
from skimage.morphology import skeletonize
from scipy.ndimage import binary_dilation


pcd_path = Path("modular_polygon_generation/libcore/data/maps/area_1.pcd")

In [46]:
pcd = o3d.io.read_point_cloud(pcd_path)
if pcd.is_empty():
    raise ValueError(f"Failed to load point cloud from {pcd_path}")

In [47]:
# rasterize the point cloud into a voxel grid
voxel_size = 0.1  # size of each voxel in meters
pts = np.asarray(pcd.points)

min_bound = pts.min(axis=0) - voxel_size / 2
max_bound = pts.max(axis=0) + voxel_size / 2
dims = np.ceil((max_bound - min_bound) / voxel_size).astype(int)

voxel_grid = np.zeros(dims, dtype=int)
indices = np.floor((pts - min_bound) / voxel_size).astype(int)
voxel_grid[indices[:, 0], indices[:, 1], indices[:, 2]] = 1

print(f"Voxel grid shape: {voxel_grid.shape}")

# fill all holes:
# add a nother layer of 2s underneath the grid
voxel_grid_padded = np.pad(voxel_grid, pad_width=1, mode='constant', constant_values=2)

# if a voxel is 0 and the point below is 2, make it 2
for z in range(1, voxel_grid_padded.shape[2]):
    for y in range(voxel_grid_padded.shape[1]):
        for x in range(voxel_grid_padded.shape[0]):
            if voxel_grid_padded[x, y, z] == 0 and voxel_grid_padded[x, y, z - 1] == 2:
                voxel_grid_padded[x, y, z] = 2

# remove the padding
voxel_grid_filled = voxel_grid_padded[1:-1, 1:-1, 1:-1]


# convert all 2s to 1s
voxel_grid_filled[voxel_grid_filled == 2] = 1

# remove the upper meter
voxel_grid_result = voxel_grid_filled[:, :, :-int(2/voxel_size)]

Voxel grid shape: (248, 484, 54)


In [ ]:
# visualize the voxel grid
occupied_indices = np.argwhere(voxel_grid_result == 1)
voxel_centers = occupied_indices * voxel_size + min_bound + voxel_size / 2
pcd_voxels = o3d.geometry.PointCloud()
pcd_voxels.points = o3d.utility.Vector3dVector(voxel_centers)

o3d.visualization.draw_geometries([pcd_voxels])

In [ ]:
# create a skeleton from the voxel grid
# add a padding of 1s around the grid
voxel_grid_padded = np.pad(voxel_grid_result, pad_width=1, mode='constant', constant_values=1)
# smooth the grid by dilating the 0s
voxel_grid_padded = binary_dilation(voxel_grid_padded == 1, iterations=2, structure=np.ones((3, 3, 3))).astype(int)

skeleton = skeletonize(voxel_grid_padded == 0, method='lee')
print(f"Skeleton shape: {skeleton.shape}")

# skeletonize a second time to thin it out more
skeleton = skeletonize(skeleton, method='lee')

# visualize the skeleton
skeleton_indices = np.argwhere(skeleton == 1)
skeleton_centers = skeleton_indices * voxel_size + min_bound + voxel_size / 2
pcd_skeleton = o3d.geometry.PointCloud()
pcd_skeleton.points = o3d.utility.Vector3dVector(skeleton_centers)

o3d.visualization.draw_geometries([pcd_skeleton])

Skeleton shape: (250, 486, 36)


In [58]:
np.sum(voxel_grid_result == 0)

np.int64(1903525)

In [59]:
np.sum(skeleton == 1)

np.int64(7308)